In [7]:
from pprint import pprint as pp
import pymongo
from pymongo import MongoClient
import json

In [8]:
client = MongoClient('localhost', 27017)
db = client['restaurant']

In [11]:
with open('./fichiersApprenants/transactions.json') as f:
    if db['transactions']:
        db['transactions'].drop()
    transactions = json.load(f)
    db.transactions.insert_many(transactions)
print('Success insertion') 

Success insertion


In [12]:
for collection in db.list_collection_names():
    print(collection)

transactions


In [14]:
db.transactions.find_one()

{'_id': ObjectId('60103f19f8f41f88b0f0ee99'),
 'Id': 100,
 'Name': 'John',
 'TransactionId': 'tran1',
 'Panier': [{'ItemId': 'a100', 'price': 200},
  {'ItemId': 'a110', 'price': 210}],
 'Subscriber': True,
 'Payment': {'Type': 'Credit-Card', 'Total': 410, 'Success': True},
 'Note': '1st Complete Record'}

# Q : La répartition des restaurants par quartier ?

In [7]:
for el in db.restaurants.aggregate([{"$group": {"_id": "$borough", "nb_restaurant": { "$sum": 1 }}}]):
    pp(el)

{'_id': 'Staten Island', 'nb_restaurant': 969}
{'_id': 'Manhattan', 'nb_restaurant': 10258}
{'_id': 'Bronx', 'nb_restaurant': 2338}
{'_id': 'Brooklyn', 'nb_restaurant': 6085}
{'_id': 'Queens', 'nb_restaurant': 5656}
{'_id': 'Missing', 'nb_restaurant': 51}


# Q : La répartition des restaurants dont la dernière inspection a donné un grade "C" par quartier ?

In [26]:
for el in db.restaurants.aggregate([{"$match": { "grades.0.grade": "C" } },{"$group": {"_id": "$borough", "nb_restaurant": { "$sum": 1 }}}]):
    pp(el)

{'_id': 'Brooklyn', 'nb_restaurant': 56}
{'_id': 'Queens', 'nb_restaurant': 47}
{'_id': 'Bronx', 'nb_restaurant': 27}
{'_id': 'Staten Island', 'nb_restaurant': 7}
{'_id': 'Manhattan', 'nb_restaurant': 83}


# Q : Calculer le score moyen des resto par quartier et trier par score décroissant ?

In [112]:
for el in db.restaurants.aggregate([ {"$unwind": "$grades" }, {"$group": {"_id": "$borough", "moyenne": { "$avg": "$grades.score" } }}, {"$sort": {"moyenne": -1}}]):
    pp(el)

{'_id': 'Queens', 'moyenne': 11.634865110930088}
{'_id': 'Brooklyn', 'moyenne': 11.447723132969035}
{'_id': 'Manhattan', 'moyenne': 11.41823125728344}
{'_id': 'Staten Island', 'moyenne': 11.370957711442786}
{'_id': 'Bronx', 'moyenne': 11.036186099942562}
{'_id': 'Missing', 'moyenne': 9.632911392405063}


Q-1. Calculer le montant total des transactions ?

In [16]:
for doc in db.transactions.aggregate([ {"$unwind": "$Panier"}, {"$group": {"_id": "Total", "somme": { "$sum": "$Panier.price" } }}]):
    pp(doc)

{'_id': 'Total', 'somme': 1640}


Q-2. Calculer le montant total par transaction ?

In [118]:
for el in db.transactions.aggregate([ {"$unwind": "$Transaction" }, {"$group": {"_id": "$TransactionId", "somme": { "$sum": "$Transaction.price" } }}]):
    pp(el)

{'_id': 'tran4', 'somme': 410}
{'_id': 'tran1', 'somme': 410}
{'_id': 'tran2', 'somme': 410}
{'_id': 'tran3', 'somme': 410}


Q-3. Calculate total payments (Payment.Total) for each payment type (Payment.Type) ?

In [124]:
for el in db.transactions.aggregate([ {"$unwind": "$Transaction" }, {"$group": {"_id": "$Payment.Type", "somme": { "$sum": "$Transaction.price" } }}]):
    pp(el)

{'_id': 'Debit-Card', 'somme': 410}
{'_id': None, 'somme': 410}
{'_id': 'Credit-Card', 'somme': 820}


Q-4. Trouver l'Id le plus élevé.

In [138]:
pp(db.transactions.find({}).sort("Id", -1)[0])

{'Id': 103,
 'Name': 'Dylan',
 'Note': 'Payment is Null',
 'Payment': None,
 'Subscriber': True,
 'Transaction': [{'ItemId': 'a100', 'price': 200},
                 {'ItemId': 'a110', 'price': 210}],
 'TransactionId': 'tran4',
 '_id': ObjectId('600eb8fd7263e84cff3fac2a')}


Q-5. Find the max price (Transaction.price) ?

In [123]:
pp(db.transactions.find({"Transaction.price"})[0])

TypeError: filter must be an instance of dict, bson.son.SON, or any other type that inherits from collections.Mapping